<a href="https://colab.research.google.com/github/ulissesbcorrea/ABSA-PyTorch/blob/master/TripAdvisor's_Dataset_Annotators_Disagreement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Libraries

In [0]:
!sudo pip3 -qqq install --upgrade disagree

In [0]:
import csv
import pandas as pd
import math
import numpy as np
import random

#Download Data

In [0]:
!gdown https://drive.google.com/uc?id=1kASaaTW6NsvMgJ5UUpbb2f2MgL8mrD-c -O anotacoes.csv

Downloading...
From: https://drive.google.com/uc?id=1kASaaTW6NsvMgJ5UUpbb2f2MgL8mrD-c
To: /content/anotacoes.csv
5.60MB [00:00, 49.2MB/s]


#Load Data 

In [0]:
anotacoesDF = pd.read_csv('anotacoes.csv', header=None, skiprows=1)
names = ['id', 'review', 'aspect', 'subaspect', 'subsubaspect', 'pol', 'impexp', 'termo_impexp', 'anotador']
anotacoesDF.columns = names

In [65]:
anotacoesDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17045 entries, 0 to 20695
Data columns (total 9 columns):
id              17045 non-null int64
review          17045 non-null object
aspect          17045 non-null object
subaspect       17044 non-null object
subsubaspect    17044 non-null object
pol             17045 non-null int64
impexp          17045 non-null object
termo_impexp    4000 non-null object
anotador        17045 non-null int64
dtypes: int64(3), object(6)
memory usage: 1.3+ MB


In [53]:
anotacoesDF.describe()

,id,pol,anotador
count,20696.000000,20696.000000,20696.000000
mean,16272.102677,0.633407,11.735263
std,6063.237558,0.685531,3.277372
min,5836.000000,-1.000000,0.000000
25%,11009.750000,1.000000,9.000000
50%,16190.500000,1.000000,10.000000
75%,21567.250000,1.000000,14.000000
max,26741.000000,1.000000,19.000000


## Limpando anotadores irregulares


Aluno que só carregou os textos e salvou, sem marcar as polaridades das opiniões.

In [0]:
anotacoes_lucas_remover =  anotacoesDF[anotacoesDF.anotador == 10].index
anotacoesDF = anotacoesDF.drop(anotacoes_lucas_remover)

Quando a sessão expira o sistema salva a anotação como user id 0.

In [0]:
anotacoes_id0_remover =  anotacoesDF[anotacoesDF.anotador == 0].index
anotacoesDF = anotacoesDF.drop(anotacoes_id0_remover)

## Número de Anotações por Anotador

In [67]:
anotacoesDF.anotador.value_counts()

8     3772
9     3221
14    2803
13    2387
15    1544
19    1163
17     729
12     589
16     521
11     211
18     105
Name: anotador, dtype: int64

## Selecionando apenas opiniões explícitas

In [0]:
anotacoesDF = anotacoesDF[anotacoesDF.impexp == 'Explícito']
# anotacoesDF.groupby(by=['review', 'aspect'])

In [69]:
anotacoesDF.describe()

,id,pol,anotador
count,13933.000000,13933.000000,13933.000000
mean,15395.779014,0.573602,11.556090
std,6598.471676,0.724543,3.460156
min,5836.000000,-1.000000,8.000000
25%,9401.000000,0.000000,8.000000
50%,13068.000000,1.000000,11.000000
75%,21542.000000,1.000000,14.000000
max,26741.000000,1.000000,19.000000


In [0]:
anotacoesDF_Explicitas = anotacoesDF.drop(columns = ['impexp', 'termo_impexp'])

# Analisando Anotações

In [71]:
anotacoesDF_Explicitas.head()

,id,review,aspect,subaspect,subsubaspect,pol,anotador
0,6431,O Hotel esta muito bem localizado os apartamen...,estabelecimento,apartamento,Selecione uma Opção,1,8
1,6430,O Hotel esta muito bem localizado os apartamen...,quarto,Selecione uma Opção,SubSubFeature,1,8
2,6429,O Hotel esta muito bem localizado os apartamen...,localização,Selecione uma Opção,SubSubFeature,1,8
3,6428,Acabei de voltar de Paris e fiquei 3 dias hosp...,estabelecimento,hotel,Selecione uma Opção,1,8
4,6427,Acabei de voltar de Paris e fiquei 3 dias hosp...,estabelecimento,apartamento,Selecione uma Opção,1,8


Número Total de *Reviews* Anotados

In [84]:
len(anotacoesDF_Explicitas.review.unique())

1496

### Contagem de Anotações por *Review*

In [0]:
temp = anotacoesDF_Explicitas.groupby(by=['review']).anotador.nunique()

In [0]:
temp = temp.reset_index()

Número de Textos Anotados Por Pelo Menos 3 Anotadores

In [91]:
df_anotadores_suficiente = temp[temp.anotador >= 3]
len(df_anotadores_suficiente)

461

In [0]:
# anotacoesDF_Explicitas.loc[anotacoesDF_Explicitas.review.str.contains('òtima Localização, próximo as estações ')]

In [0]:
# anotacoesDF_Explicitas.groupby(by=['review', 'aspect', 'subaspect', 'subsubaspect']).anotador.nunique()

In [0]:
# temp = temp.reset_index()
# temp[temp.anotador>=2].head()

# Preparing for disagreement

In [0]:
anotador1 = 8
anotador2 = 9
label_anotador1 = 'anotador_'+str(anotador1)
label_anotador2 = 'anotador_'+str(anotador2)

anotacoes_1 = anotacoesDF_Explicitas[anotacoesDF_Explicitas.anotador == anotador1].copy()
anotacoes_2 = anotacoesDF_Explicitas[anotacoesDF_Explicitas.anotador == anotador2].copy()

In [0]:
merged = pd.merge(left=anotacoes_1, right=anotacoes_2, on=['review', 'aspect', 'subaspect', 'subsubaspect'], suffixes=('_'+str(anotador1),'_'+str(anotador2)))

In [0]:
merged_cleaned = merged.drop(columns=['review', 'aspect', 'subaspect', 'subsubaspect', 'id_'+str(anotador1), 'id_'+str(anotador2), label_anotador1, label_anotador2], axis = 1)

In [0]:
from disagree import BiDisagreements
from disagree.metrics import Metrics, Krippendorff

In [0]:
labels = [-1, 0, 1]

In [0]:
bidis = BiDisagreements(merged_cleaned, labels)

In [0]:
bidis.agreements_summary()

In [0]:
mat = bidis.agreements_matrix()
mat_normalised = bidis.agreements_matrix(normalise=True)

In [0]:
print(mat)
print(mat_normalised)

In [0]:
mets = Metrics(merged_cleaned, labels)

In [0]:
mets.cohens_kappa('pol_'+str(anotador1),'pol_'+str(anotador2))

In [0]:
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(merged_cleaned['pol_'+str(anotador1)], merged_cleaned['pol_'+str(anotador2)])

In [0]:
from sklearn.metrics import confusion_matrix
confusion_matrix(merged_cleaned['pol'+str(anotador1)], merged_cleaned['pol'+str(anotador2)])

In [0]:
len(merged_cleaned[merged_cleaned['pol'+str(anotador1)] == 1])

In [0]:
len(merged_cleaned[merged_cleaned['pol'+str(anotador2)]== 1])

In [0]:
merged.loc[merged_cleaned[merged_cleaned['pol'+str(anotador1)] != merged_cleaned['pol'+str(anotador2)]].index]

In [0]:
def locate_review(df, txt):
  return df[df.review.str.contains(txt, regex = False)].review[df[df.review.str.contains(txt, regex = False)].index[0]]

In [0]:
locate_review(anotacoesDF_Explicitas, 'Hotel aconchegante, pessoal simpático')

In [0]:
anotacoesDF_Explicitas.head()

In [0]:
anotacoesDF_Explicitas[anotacoesDF_Explicitas.anotador == anotador1].pol.value_counts()

In [0]:
anotacoesDF_Explicitas.head()

In [0]:
grouped = anotacoesDF_Explicitas.groupby(by=['review']).anotador.nunique()

In [0]:
grouped = grouped.reset_index()
len(grouped[grouped.anotador >= 2])


In [0]:
grouped.head()

In [0]:
def locate_review_id(df, txt):
  return df[df.review.str.contains(txt, regex = False)].id.head(1).index[0]

In [0]:
ids = []
for index,row in grouped.iterrows(): 
  # print(row['review'])
  review = row['review']
  ids.append(locate_review_id(anotacoesDF_Explicitas, review))

In [0]:
grouped['index'] = ids

In [0]:
anotacoesDF_Explicitas.anotador.value_counts()

In [0]:
anotacoesDF_Explicitas[anotacoesDF_Explicitas.anotador == 8].review.nunique()

In [0]:
anotacoesDF_Explicitas[anotacoesDF_Explicitas.anotador == 9].review.nunique()

In [0]:
anotacoesDF_Explicitas[anotacoesDF_Explicitas.anotador == 12].review.nunique()

In [0]:
anotacoesDF_Explicitas[anotacoesDF_Explicitas.anotador == 11].review.nunique()

In [0]:
anotacoesDF_Explicitas[anotacoesDF_Explicitas.anotador == 0].review.nunique()

In [0]:
anotacoesDF_Explicitas[anotacoesDF_Explicitas.anotador == 13].review.nunique()

In [0]:
!ls

In [0]:
def save_row(row):
  # print(row)
  anotador = str(row['anotador'])
  index = str(row['id'])
  # print(anotador+'_'+index+ '.txt')
  with open(anotador+'_'+index+ '.txt', 'w') as f:
    f.write(row['review'])

# _ = anotacoesDF_Explicitas[anotacoesDF_Explicitas.anotador == 9].apply(save_row, axis=1)

In [0]:
!rm *.txt

In [0]:
anotacoes_anotador = anotacoesDF_Explicitas[anotacoesDF_Explicitas.anotador == 8].drop_duplicates(subset=['review'])
print(len(anotacoes_anotador), len(anotacoesDF_Explicitas[anotacoesDF_Explicitas.anotador == 8]))

In [0]:
anotacoes_anotador[anotacoes_anotador.review.str.contains('O hotel é simples e as acomodações pequenas, mas bem limpo')]

In [0]:
anotacoes_anotador.groupby(by=['review']).pol.value_counts()

In [0]:
def salva_anotados(df):
  anotadores = df.anotador.unique()
  for anotador in anotadores:
    print(f'Anotador:{anotador}')
    anotacoes_anotador = df[df.anotador == anotador].drop_duplicates(subset=['review']).copy()
    print(len(anotacoes_anotador))
    anotacoes_anotador.apply(save_row, axis=1)
salva_anotados(anotacoesDF_Explicitas)

In [0]:
!ls

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# !mkdir -p drive/My\ Drive/reviews/8
# !cp 8*.txt drive/My\ Drive/reviews/8

In [0]:
anotadores = anotacoesDF_Explicitas.anotador.unique()
anotadores = [9]
for anotador in anotadores:
  print(f'Anotador: {anotador}')
  !mkdir -p drive/My\ Drive/reviews/{anotador}
  !cp -v {anotador}*.txt drive/My\ Drive/reviews/{anotador}

In [0]:
!l drive/My\ Drive/reviews/s9| wc -l

In [0]:
!mkdir reviews9
!mv 9*.txt reviews9

In [0]:
!zip -r reviews9.zip reviews9

In [0]:
from google.colab import files

files.download('reviews9.zip')

In [0]:
 !cp reviews9.zip drive/My\ Drive/reviews/


In [0]:
from nltk import agreement
rater1 = [1,1,1]
rater2 = [1,1,0]
rater3 = [0,1,1]

taskdata=[[0,str(i),str(rater1[i])] for i in range(0,len(rater1))]+[[1,str(i),str(rater2[i])] for i in range(0,len(rater2))]+[[2,str(i),str(rater3[i])] for i in range(0,len(rater3))]
ratingtask = agreement.AnnotationTask(data=taskdata)
print("kappa " +str(ratingtask.kappa()))
print("fleiss " + str(ratingtask.multi_kappa()))
print("alpha " +str(ratingtask.alpha()))
print("scotts " + str(ratingtask.pi()))